In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# load data into dataframes
sla_df = pd.read_excel('data/SLA_matrix.xlsx', engine='openpyxl')
orders_df = pd.read_csv('data/delivery_orders_march.csv')

In [3]:
sla_df

,1st Attempt SLA\n(Working Days),Unnamed: 1,Destination (Buyer),Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,Metro Manila,Luzon,Visayas,Mindanao
1,Origin\n(Seller),Metro Manila,3 working days,5 working days,7 working days,7 working days
2,NaN,Luzon,5 working days,5 working days,7 working days,7 working days
3,NaN,Visayas,7 working days,7 working days,7 working days,7 working days
4,NaN,Mindanao,7 working days,7 working days,7 working days,7 working days
5,NaN,NaN,NaN,NaN,NaN,NaN
6,"Working Days are defined as Mon - Sat, Excludi...",NaN,NaN,NaN,NaN,NaN
7,SLA calculation begins from the next day after...,NaN,NaN,NaN,NaN,NaN
8,2nd Attempt must be no later than 3 working da...,NaN,NaN,NaN,NaN,NaN


In [4]:
orders_df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [5]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


In [6]:
# transform pick, 1st_deliver_attempt and 2nd_deliver_attempt
date_cols = ['pick', '1st_deliver_attempt', '2nd_deliver_attempt']

orders_df[date_cols] += 8 * 60 * 60 # local time is GMT +8

orders_df['2nd_deliver_attempt'] = orders_df['2nd_deliver_attempt'].replace(np.nan, 0)

for col in date_cols:
    orders_df[col] = pd.to_datetime(orders_df[col], unit='s').dt.date

orders_df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [7]:
# extract states from buyeraddress and selleraddress
states = ['metro manila', 'luzon', 'visayas', 'mindanao']
orders_df['buyeraddress'] = orders_df['buyeraddress'].str.lower()
orders_df['selleraddress']  = orders_df['selleraddress'].str.lower()
for state in states:
    orders_df.loc[orders_df['buyeraddress'].str.contains(state), 'buyeraddress'] = state
    orders_df.loc[orders_df['selleraddress'].str.contains(state), 'selleraddress'] = state
orders_df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,metro manila,metro manila
1,2219624609,2020-03-04,2020-03-06,2020-03-10,metro manila,metro manila
2,2220979489,2020-03-04,2020-03-06,1970-01-01,metro manila,metro manila
3,2221066352,2020-03-05,2020-03-07,1970-01-01,metro manila,metro manila
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,metro manila


In [8]:
# get maximum number of working days before the orders are deemed as late
def get_max_days(row):
    if row['buyeraddress'] == 'metro manila' and row['selleraddress'] == 'metro manila':
        return 3
    elif (row['buyeraddress'] == 'metro manila' or row['buyeraddress'] == 'luzon') and (row['selleraddress'] == 'metro manila' or  row['selleraddress'] == 'luzon'):
        return 5
    else:
        return 7

orders_df['max_days'] = orders_df.apply(lambda row: get_max_days(row), axis=1)
orders_df.head(15)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,max_days
0,2215676524,2020-03-02,2020-03-05,1970-01-01,metro manila,metro manila,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,metro manila,metro manila,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,metro manila,metro manila,3
3,2221066352,2020-03-05,2020-03-07,1970-01-01,metro manila,metro manila,3
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,metro manila,5
5,2222597288,2020-03-04,2020-03-07,1970-01-01,metro manila,metro manila,3
6,2222738456,2020-03-02,2020-03-05,2020-03-09,metro manila,metro manila,3
7,2224695304,2020-03-02,2020-03-10,1970-01-01,metro manila,metro manila,3
8,2224704587,2020-03-04,2020-03-05,2020-03-09,luzon,metro manila,5
9,2225138267,2020-03-04,2020-03-10,1970-01-01,visayas,metro manila,7


In [9]:
# calculate number of working days taken to deliver the order
holidays = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']
orders_list = orders_df.to_dict(orient='records')
for order in orders_list:
    order['1st_deliver_attempt_days'] = np.busday_count(order['pick'], order['1st_deliver_attempt'], weekmask='1111110', holidays=holidays)
    order['2nd_deliver_attempt_days'] = np.busday_count(order['1st_deliver_attempt'], order['2nd_deliver_attempt'], weekmask='1111110', holidays=holidays)
orders_df = pd.DataFrame(orders_list)
orders_df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,max_days,1st_deliver_attempt_days,2nd_deliver_attempt_days
0,2215676524,2020-03-02,2020-03-05,1970-01-01,metro manila,metro manila,3,3,-15708
1,2219624609,2020-03-04,2020-03-06,2020-03-10,metro manila,metro manila,3,2,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,metro manila,metro manila,3,2,-15709
3,2221066352,2020-03-05,2020-03-07,1970-01-01,metro manila,metro manila,3,2,-15710
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,metro manila,5,2,-15709


In [10]:
# determine whether orders are late
orders_df['is_late'] = (orders_df['1st_deliver_attempt_days'] > orders_df['max_days']) | (orders_df['2nd_deliver_attempt_days'] > 3)
orders_df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,max_days,1st_deliver_attempt_days,2nd_deliver_attempt_days,is_late
0,2215676524,2020-03-02,2020-03-05,1970-01-01,metro manila,metro manila,3,3,-15708,False
1,2219624609,2020-03-04,2020-03-06,2020-03-10,metro manila,metro manila,3,2,3,False
2,2220979489,2020-03-04,2020-03-06,1970-01-01,metro manila,metro manila,3,2,-15709,False
3,2221066352,2020-03-05,2020-03-07,1970-01-01,metro manila,metro manila,3,2,-15710,False
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,metro manila,5,2,-15709,False


In [11]:
# format result
result_df = pd.DataFrame({'orderid': orders_df['orderid'], 'is_late': orders_df['is_late'].apply(int)})
result_df.head(10)

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
5,2222597288,0
6,2222738456,0
7,2224695304,1
8,2224704587,0
9,2225138267,0


In [12]:
# save result
result_df.to_csv('output/result.csv', index=False)
print(len(result_df))

3176313
